In [ ]:
### Sub sampling
#Want to create a polars frame with the data
import polars as pl
import Utils.constants
import importlib
import Utils.visualisation as vis
importlib.reload(vis) 

from Utils.constants import FOLDER
from Utils.constants import N_LANES, N_SPEEDS, N_TIME_LIMIT, N_DIRECTIONS, N_SPEED_DEVIATION, N_LANE_DEVIATION, N_TIME_LIMIT
from Utils.constants import MAX_COST_VALUE,INDEX_SPEED_MULTIPLIER
from Utils.data_generation import track_data_import, lane_creation, track_thinning
from Utils.data_generation import load_pl_data, load_np_data

import Utils.data_generation as data_gen
import Utils.path_optimisation as path_opt
import Utils.visualisation as vis
import Utils.book_keeping as bk

In [ ]:
#from Utils.path_optimisation import processing_optimal_trajectory
import importlib
importlib.reload(data_gen) 
opt_data_path,(av_path,acc_path,tc_path) = data_gen.data_generation("Run1_df")

In [ ]:
#from Utils.path_optimisation import processing_optimal_trajectory
import Utils.data_generation as data_gen
importlib.reload(data_gen) 

#get the data
# opt_data_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Run1_df_l11.parquet'
# av_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Data/av_11'
# acc_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Data/acc_11'
# tc_path = 'C:/Users/johan/OneDrive/Documents/Python/Python/SilverStone/Data/tc_11'
opt_df = data_gen.load_pl_data(opt_data_path)
acc_arr = data_gen.load_np_data(acc_path)
av_arr = data_gen.load_np_data(av_path)
tc_arr = data_gen.load_np_data(tc_path)

In [ ]:
#from Utils.path_optimisation import processing_optimal_trajectory
importlib.reload(path_opt) 
importlib.reload(data_gen) 

# F_MAX_ACC = 20000 #8750
# F_MAX_T = 8750
# MASS = 1000 #typical weihgt of a F1 car 1000 kg
# tau_h = 0.1
# tau_v = 1.0
# g = 10
# mu = 1.0

#F_MAX_ACC = 12000 #8750
F_MAX_ACC = 25000
F_X_NEG_SCALE = 0.3
F_X_MAX_SCALE = 0.1
F_Y_MAX_SCALE = 1.0
MASS = 800 #typical weight of a F1 car 1000 kg
tau_h = 0.62
tau_v = 5.4
g = 10
mu = 1.0
path_opt.add_Force_requiremments(MASS, g, tau_h, tau_v, mu, F_MAX_ACC)


In [ ]:
#cost data used later
cost_df = path_opt.make_cost_df(opt_df,MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE)

In [ ]:
importlib.reload(path_opt) 
importlib.reload(data_gen) 
importlib.reload(vis) 
importlib.reload(bk) 

#F_MAX_ACC = 12000 #8750
F_MAX_ACC = 25000
F_X_NEG_SCALE = 0.3
F_X_MAX_SCALE = 0.1
F_Y_MAX_SCALE = 1.0
MASS = 800 #typical weight of a F1 car 1000 kg
tau_h = 0.62
tau_v = 5.4
g = 10
mu = 1.0

rho_range = [0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.0]
force_range = [25000, 23000, 21000, 19000, 17000, 15000, 13000]

In [ ]:
for f_max in force_range: 
    F_MAX_ACC = f_max
    cost_df = path_opt.make_cost_df(opt_df,MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE)
    for rho in rho_range:
        path_opt.run_opt(cost_df, MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE, rho)

In [ ]:
F_MAX_ACC = 25000
cost_df = make_cost_df(opt_df,MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE)
for rho in rho_range:
    run_opt(cost_df, MASS, g, tau_h, tau_v, mu, F_MAX_ACC, F_X_NEG_SCALE, F_X_MAX_SCALE, F_Y_MAX_SCALE, rho)